## 24. Functions

Like "real" programming languages, Bash has functions, though in a somewhat limited implementation. A function is a subroutine, a code block that implements a set of operations, a "black box" that performs a specified task. Wherever there is repetitive code, when a task repeats with only slight variations in procedure, then consider using a function.

or

This second form will cheer the hearts of C programmers (and is more portable).  
As in C, the function's opening bracket may optionally appear on the second line.

A function may be "compacted" into a single line.

In [ ]:
fun () { echo "This is a function"; echo; }

In this case, however, a semicolon must follow the final command in the function.

In [ ]:
fun () { echo "This is a function"; echo } # Error!
fun2 () { echo "Even a single-command function? Yes!"; } # Right!

Functions are called, triggered, simply by invoking their names.   
A function call is equivalent to a command.

#### Example 24-1. Simple functions

In [2]:
cat ex59.sh

#!/bin/bash
# ex59.sh: Exercising functions (simple).

JUST_A_SECOND=1

funky ()
{ # This is about as simple as functions get.
  echo "This is a funky function."
  echo "Now exiting funky function."
} # Function declaration must precede call.

fun ()
{ # A somewhat more complex function.
  i=0
  REPEATS=5
  echo
  echo "And now the fun really begins."
  echo
  sleep $JUST_A_SECOND   # Hey, wait a second!
  while [ $i -lt $REPEATS ]
  do
    echo "----------FUNCTIONS---------->"
    echo "<------------ARE-------------"
    echo "<------------FUN------------>"
    echo
    let "i+=1"
  done
}

# Now, call the functions.
funky
fun

exit $?


In [3]:
bash ex59.sh

This is a funky function.
Now exiting funky function.

And now the fun really begins.

----------FUNCTIONS---------->
<------------ARE-------------
<------------FUN------------>

----------FUNCTIONS---------->
<------------ARE-------------
<------------FUN------------>

----------FUNCTIONS---------->
<------------ARE-------------
<------------FUN------------>

----------FUNCTIONS---------->
<------------ARE-------------
<------------FUN------------>

----------FUNCTIONS---------->
<------------ARE-------------
<------------FUN------------>



The function definition must precede the first call to it.   
There is no method of "declaring" the function, as, for example, in C.

In [4]:
# Will give an error message, since function "f1" not yet defined.
f1

f1: command not found


In [5]:
# This doesn't help either.
# Still an error message.
declare -f f1
f1

f1: command not found


In [6]:
# However...
f1 ()
{
  echo "Calling function \"f2\" from within function \"f1\"."
  f2
}

f2 ()
{
  echo "Function \"f2\"."
}
#  Function "f2" is not actually called until this point,
#+ although it is referenced before its definition.
#  This is permissible.
f1

Calling function "f2" from within function "f1".
Function "f2".


Functions may not be empty!

In [7]:
cat empty-function.sh

#!/bin/bash
# empty-function.sh

empty ()
{
}

exit 0   # Will not exit here!


In [10]:
sh empty-function.sh;echo $?

empty-function.sh: 6: empty-function.sh: Syntax error: "}" unexpected
2


In [11]:
# Note that a function containing only comments is empty.
# Results in same error message as above.
func ()
{
  # Comment 1.
  # Comment 2.
  # This is still an empty function.
  # Thank you, Mark Bova, for pointing this out.
}

func

bash: syntax error near unexpected token `}'
No command 'func' found, did you mean:
 Command 'pfunc' from package 'libmodule-info-perl' (universe)
func: command not found


In [ ]:
# However ...

not_quite_empty ()
{
  illegal_command
}   #  A script containing this function will *not* bomb
    #+ as long as the function is not called.
    
not_empty ()
{
  :
}   # Contains a : (null command), and this is okay.

It is even possible to nest a function within another function, although this is not very useful.

In [17]:
cat nested-function.sh

#!/bin/bash

f1 ()
{
  f2 () # nested
  {
   	echo "Function \"f2\", inside \"f1\"."
  }
}
f2  # Gives an error message.
    # Even a preceding "declare -f f2" wouldn't help.

echo

# Does nothing, since calling "f1" does not automatically call "f2".
f1 
#  Now, it's all right to call "f2",
#+ since its definition has been made visible by calling "f1".
f2 



In [18]:
sh nested-function.sh

nested-function.sh: 10: nested-function.sh: f2: not found

Function "f2", inside "f1".


Function declarations can appear in unlikely places,   
even where a command would otherwise go.

In [ ]:
# Permissible, but useless.
ls -l | foo() { echo "foo"; }

In [22]:
if [ "$USER" = liheyi ]
then
  liheyi_greet () # Function definition embedded in an if/then construct.
  {
    echo "Hello, liheyi."
  }
fi

# Works only for liheyi, and other users get an error.
liheyi_greet

Hello, liheyi.


In [23]:
# Something like this might be useful in some contexts.
NO_EXIT=1      # Will enable function definition below.

[[ $NO_EXIT -eq 1 ]] && exit() { true; }   # Function definition in an "and-list".
# If $NO_EXIT is 1, declares "exit ()".
# This disables the "exit" builtin by aliasing it to "true".

exit           # Invokes "exit ()" function, not "exit" builtin.

In [24]:
# Or, similarly:
filename=file1
[ -f "$filename" ] &&
foo () { rm -f "$filename"; echo "File "$filename" deleted."; } ||
foo () { echo "File "$filename" not found."; touch bar; }
foo

File file1 not found.


Function names can take strange forms.

In [25]:
   _(){ for i in {1..10}; do echo -n "$FUNCNAME"; done; echo; }
# ^^^            No space between function name and parentheses.
#                This doesn't always work. Why not?

# Now, let's invoke the function.
_          # __________
#           ^^^^^^^^^^  10 underscores (10 x function name)!
# A "naked" underscore is an acceptable function name.

__________


In [26]:
# In fact, a colon is likewise an acceptable function name.
:(){ echo ":"; }; :
# Of what use is this?
# It's a devious way to obfuscate the code in a script.

:


What happens when different versions of the same function appear in a script?

In [27]:
cat different_version.sh

#!/bin/bash
# As Yan Chen points out,
# when a function is defined multiple times,
# the final version is what is invoked.
# This is not, however, particularly useful.

func ()
{
	echo "First version of func ()."
}

func ()
{
	echo "Second version of func ()."
}

func  # Second version of func ().

exit $?

#  It is even possible to use functions to override
#+ or preempt system commands.
#  Of course, this is *not* advisable.


In [28]:
bash different_version.sh

Second version of func ().


### Complex Functions and Function Complexities

Functions may process arguments passed to them and return an exit status to the script for further processing.

The function refers to the passed arguments by position  
(as if they were positional parameters), that is, \$1,\$2, and so forth.

#### Example 24-2. Function Taking Parameters

In [29]:
cat fun_take_para.sh

#!/bin/bash
# Functions and parameters

DEFAULT=default    # Default param value.

func2 () {
  if [ -z "$1" ] # Is parameter #1 zero length?
  then
    echo "-Parameter #1 is zero length.-" # Or no parameter passed.
  else
    echo "-Parameter #1 is \"$1\".-"
  fi

  variable=${1-$DEFAULT}         #  What does
  echo "variable = $variable"    #+ parameter substitution show?
                                 # ---------------------------
                                 # It distinguishes between
  if [ "$2" ]
  then
    echo "-Parameter #2 is \"$2\".-"
  fi

  return 0
}

echo

echo "Nothing passed."
func2                        # Called with no params
echo

echo "Zero-length parameter passed."
func2 ""                     # Called with zero-length param
echo

echo "Null parameter passed."
func2 "$uninitialized_param" # Called with uninitialized param
echo

echo "One parameter passed."
func2 first                  # Called with one param
echo

echo "Two parameters passed."
func2 first 

In [30]:
bash fun_take_para.sh


Nothing passed.
-Parameter #1 is zero length.-
variable = default

Zero-length parameter passed.
-Parameter #1 is zero length.-
variable = 

Null parameter passed.
-Parameter #1 is zero length.-
variable = 

One parameter passed.
-Parameter #1 is "first".-
variable = first

Two parameters passed.
-Parameter #1 is "first".-
variable = first
-Parameter #2 is "second".-

"" "second" passed.
-Parameter #1 is zero length.-
variable = 
-Parameter #2 is "second".-



The shift command works on arguments passed to functions (see Example 36-18).  
But, what about command-line arguments passed to the script?   
Does a function see them? Well, let's clear up the confusion.

#### Example 24-3. Functions and command-line args passed to the script

In [31]:
cat func-cmdlinearg.sh

#!/bin/bash
# func-cmdlinearg.sh
#  Call this script with a command-line argument,
#+ something like $0 arg1.

func ()
{
  echo "$1" # Echoes first arg passed to the function.
} # Does a command-line arg qualify?

echo "First call to function: no arg passed."
echo "See if command-line arg is seen."
func
# No! Command-line arg not seen.

echo "============================================================"
echo
echo "Second call to function: command-line arg passed explicitly."
func $1
# Now it's seen!

exit 0


In [33]:
bash func-cmdlinearg.sh liheyi

First call to function: no arg passed.
See if command-line arg is seen.


Second call to function: command-line arg passed explicitly.
liheyi


In contrast to certain other programming languages, shell scripts normally pass only value parameters to functions. Variable names (which are actually pointers), if passed as parameters to functions, will be treated as string literals. Functions interpret their arguments literally.

Indirect variable references (see Example 37-2) provide a clumsy sort of mechanism for passing variable pointers to functions.

#### Example 24-4. Passing an indirect reference to a function

In [34]:
cat ind-func.sh

#!/bin/bash
# ind-func.sh: Passing an indirect reference to a function.

echo_var ()
{
  echo "$1"
}

message=Hello
Hello=Goodbye

echo_var "$message"       # Hello
# Now, let's pass an indirect reference to the function.
echo_var "${!message}"    # Goodbye

echo "-------------"

# What happens if we change the contents of "hello" variable?
Hello="Hello, again!"
echo_var "$message"       # Hello
echo_var "${!message}"    # Hello, again!

exit 0


In [35]:
bash ind-func.sh

Hello
Goodbye
-------------
Hello
Hello, again!


The next logical question is whether parameters can be dereferenced after being passed to a function.

#### Example 24-5. Dereferencing a parameter passed to a function

In [36]:
cat dereference.sh

#!/bin/bash
# dereference.sh
# Dereferencing parameter passed to a function.

dereference ()
{
  y=\$"$1"     # Name of variable (not value!).
  echo $y      # $Junk
  x=`eval "expr \"$y\" "`
  echo $1=$x
  eval "$1=\"Some Different Text \"" # Assign new value.
}

Junk="Some Text"
echo $Junk "before"   # Some Text before

dereference Junk
echo $Junk "after"    # Some Different Text after

exit 0


In [37]:
bash dereference.sh

Some Text before
$Junk
Junk=Some Text
Some Different Text after


#### Example 24-6. Again, dereferencing a parameter passed to a function

In [38]:
cat ref-params.sh

#!/bin/bash
# ref-params.sh: Dereferencing a parameter passed to a function.
#                (Complex Example)

ITERATIONS=3     # How many times to get input.
icount=1

my_read () {
  #  Called with my_read varname,
  #+ outputs the previous value between brackets as the default value,
  #+ then asks for a new value.

  local local_var

  echo -n "Enter a value "
  eval 'echo -n "[$'$1'] "'    #  Previous value.
  # eval echo -n "[\$$1] "     #  Easier to understand,
                               #+ but loses trailing space in user prompt.

  read local_var
  [ -n "$local_var" ] && eval $1=\$local_var

  # "And-list": if "local_var" then set "$1" to its value.
}

echo

while [ "$icount" -le "$ITERATIONS" ]
do
  my_read var
  echo "Entry #$icount = $var"
  let "icount += 1"
  echo
done

# Thanks to Stephane Chazelas for providing this instructive example.
exit 0


#### Exit and Return

##### exit status

##### return

#### Example 24-7. Maximum of two numbers

In [39]:
cat max.sh

#!/bin/bash
# max.sh: Maximum of two integers.

E_PARAM_ERR=250   # If less than 2 params passed to function.
EQUAL=251         # Return value if both params equal.
#  Error values out of range of any
#+ params that might be fed to the function.

max2 ()           # Returns larger of two numbers.
{                 # Note: numbers compared must be less than 250.
  if [ -z "$2" ]
  then
    return $E_PARAM_ERR
  fi

  if [ "$1" -eq "$2" ]
  then
    return $EQUAL
  else
    if [ "$1" -gt "$2" ]
    then
      return $1
  	else
      return $2
  	fi
  fi
}

max2 33 34
return_val=$?

if [ "$return_val" -eq $E_PARAM_ERR ]
then
  echo "Need to pass two parameters to the function."
elif [ "$return_val" -eq $EQUAL ]
  then
    echo "The two numbers are equal."
  else
    echo "The larger of the two numbers is $return_val."
fi

exit 0


In [40]:
bash max.sh

The larger of the two numbers is 34.


For a function to return a string or array, use a dedicated variable.

In [41]:
count_lines_in_etc_passwd()
{
  [[ -r /etc/passwd ]] && REPLY=$(echo $(wc -l < /etc/passwd))
  #  If /etc/passwd is readable, set REPLY to line count.
  #  Returns both a parameter value and status information.
  #  The 'echo' seems unnecessary, but . . .
  #+ it removes excess whitespace from the output.
}

if count_lines_in_etc_passwd
then
  echo "There are $REPLY lines in /etc/passwd."
else
  echo "Cannot count lines in /etc/passwd."
fi

There are 38 lines in /etc/passwd.


#### Example 24-8. Converting numbers to Roman numerals

In [1]:
cat arabic_to_roman.sh

#!/bin/bash
# Arabic number to Roman numeral conversion
# Range: 0 - 200
# It's crude, but it works.

# Extending the range and otherwise improving the script is left as an exercise.

# Usage: roman number-to-convert

LIMIT=200
E_ARG_ERR=65
E_OUT_OF_RANGE=66

if [ -z "$1" ]
then
  echo "Usage: `basename $0` number-to-convert"
  exit $E_ARG_ERR
fi

num=$1
if [ "$num" -gt $LIMIT ]
then
  echo "Out of range!"
  exit $E_OUT_OF_RANGE
fi

to_roman () # Must declare function before first call to it.
{
  number=$1
  factor=$2
  rchar=$3
  let "remainder = number - factor"
  while [ "$remainder" -ge 0 ]
  do
    echo -n $rchar
    let "number -= factor"
    let "remainder = number - factor"
  done
  return $number
  # Exercises:
  # ---------
  # 1) Explain how this function works.
  # Hint: division by successive subtraction.
  # 2) Extend to range of the function.
  # Hint: use "echo" and command-substitution capture.
}

to_roman $num 100 C
num=$?
to_roman $num 90 LXXXX
num=$?
to_roman $num 5

In [2]:
bash arabic_to_roman.sh 18

XVIII


In [3]:
bash arabic_to_roman.sh 123

CXXIII


The largest positive integer a function can return is 255. The return command is closely tied to the concept of exit status, which accounts for this particular limitation. Fortunately, there are various workarounds for those situations requiring a large integer return value from a function.

#### Example 24-9. Testing large return values in a function

In [4]:
cat return-test.sh

#!/bin/bash
# return-test.sh

# The largest positive value a function can return is 255.

return_test ()    # Returns whatever passed to it.
{
	return $1
}

return_test 27      # o.k.
echo $?             # Returns 27.

return_test 255     # Still o.k.
echo $?             # Returns 255.

return_test 257     # Error!
echo $?             # Returns 1 (return code for miscellaneous error).

# =========================================================
return_test -151896 # Do large negative numbers work?
echo $?             # Will this return -151896?
                    # No! It returns 168.
#  Version of Bash before 2.05b permitted
#+ large negative integer return values.
#  It happened to be a useful feature.
#  Newer versions of Bash unfortunately plug this loophole.
#  This may break older scripts.
#  Caution!
# =========================================================

exit 0


In [5]:
bash return-test.sh

27
255
1
168


A workaround for obtaining large integer "return values" is to simply assign the "return value" to a global variable.

In [6]:
# Global variable to hold oversize return value of function.
Return_Val=

alt_return_test ()
{
  fvar=$1
  Return_Val=$fvar
  return          # Returns 0 (success).
}

In [7]:
alt_return_test 1
echo $?                           # 0
echo "return value = $Return_Val" # 1

0
return value = 1


In [8]:
alt_return_test 256
echo "return value = $Return_Val" # 256

return value = 256


In [9]:
alt_return_test 257
echo "return value = $Return_Val" # 257

return value = 257


In [10]:
alt_return_test 25701
echo "return value = $Return_Val" #25701

return value = 25701


A more elegant method is to have the function echo its "return value to stdout," and then capture it by command substitution. See the discussion of this in Section 36.7.

#### Example 24-10. Comparing two large integers

In [11]:
cat max2.sh

#!/bin/bash
# max2.sh: Maximum of two LARGE integers.

#  This is the previous "max.sh" example,
#+ modified to permit comparing large integers.

EQUAL=0             # Return value if both params equal.
E_PARAM_ERR=-99999  # Not enough params passed to function.
#           ^^^^^^    Out of range of any params that might be passed.

max2 ()             # "Returns" larger of two numbers.
{
  if [ -z "$2" ]
  then
    echo $E_PARAM_ERR
    return
  fi

  if [ "$1" -eq "$2" ]
  then
    echo $EQUAL
    return
  else
    if [ "$1" -gt "$2" ]
    then
      retval=$1
    else
      retval=$2
    fi
  fi
  echo $retval      # Echoes (to stdout), rather than returning value.
                    # Why?
}

return_val=$(max2 33001 33997)
#            ^^^^             Function name
#                 ^^^^^ ^^^^^ Params passed
#  This is actually a form of command substitution:
#+ treating a function as if it were a command,
#+ and assigning the stdout of the function to the variable "return_val."


In [12]:
bash max2.sh

The larger of the two numbers is 33997.


Here is another example of capturing a function "return value."   
Understanding it requires some knowledge of awk.

In [13]:
month_length ()   # Takes month number as an argument.
{                 # Returns number of days in month.
monthD="31 28 31 30 31 30 31 31 30 31 30 31"  # Declare as local?
echo "$monthD" | awk '{ print $'"${1}"' }'    # Tricky.
#                            ^^^^^^^^^
# Parameter passed to function ($1 -- month number), then to awk.
# Awk sees this as "print $1 . . . print $12" (depending on month number)
# Template for passing a parameter to embedded awk script:
#                                 $'"${script_parameter}"'

#    Here's a slightly simpler awk construct:
#    echo $monthD | awk -v month=$1 '{print $(month)}'
#    Uses the -v awk option, which assigns a variable value
#+   prior to execution of the awk program block.
#    Thank you, Rich.
#  Needs error checking for correct parameter range (1-12)
#+ and for February in leap year.
}
# ----------------------------------------------
# Usage example:
month=4          # April, for example (4th month).
days_in=$(month_length $month)
echo $days_in    # 30
# ----------------------------------------------

30


Exercise:   
Using what we have just learned, extend the previous Roman numerals example to accept arbitrarily large input.

##### Redirection

Redirecting the stdin of a function  
A function is essentially a code block, which means its stdin can be redirected(as in Example 3-1).

#### Example 24-11. Real name from username

In [14]:
cat realname.sh

#!/bin/bash
# realname.sh
#
# From username, gets "real name" from /etc/passwd.

ARGCOUNT=1          # Expect one arg.
E_WRONGARGS=85

file=/etc/passwd
pattern=$1

if [ $# -ne "$ARGCOUNT" ]
then
  echo "Usage: `basename $0` USERNAME"
  exit $E_WRONGARGS
fi

file_excerpt ()     #  Scan file for pattern,
{                   #+ then print relevant portion of line.
  while read line   # "while" does not necessarily need [ condition ]
  do
    echo "$line" | grep $1 | awk -F":" '{ print $5 }'
    # Have awk use ":" delimiter.
  done
} <$file            # Redirect into function's stdin.
file_excerpt $pattern

# Yes, this entire script could be reduced to
#       grep PATTERN /etc/passwd | awk -F":" '{ print $5 }'
# or
#       awk -F: '/PATTERN/ {print $5}'
# or
#       awk -F: '($1 == "username") { print $5 }' # real name from username
# However, it might not be as instructive.

exit 0


In [15]:
bash realname.sh liheyi

liheyi,,,


In [16]:
bash realname.sh root

root


There is an alternate, and perhaps less confusing method of redirecting a function's stdin.   
This involves redirecting the stdin to an embedded bracketed code block within the function.

In [ ]:
# Instead of:
Function ()
{
  ...
} < file

# Try this:
Function ()
{
  {
    ...
  } < file
}

# Similarly,
Function () # This works.
{
  {
    echo $*
  } | tr a b
}

Function () # This doesn't work.
{
  echo $*
} | tr a b  # A nested code block is mandatory here.

### Local Variables

#### What makes a variable local?

###### local variables
A variable declared as local is one that is visible only within the block of code in which it appears.   
It has local scope.   
In a function, a local variable has meaning only within that function block.  

#### Example 24-12. Local variable visibility

In [3]:
cat ex62.sh

#!/bin/bash
# ex62.sh: Global and local variables inside a function.

func ()
{
  local loc_var=23     # Declared as local variable.
  echo                 # Uses the 'local' builtin.
  echo "\"loc_var\" in function = $loc_var"
  global_var=999       # Not declared as local.
                       # Therefore, defaults to global.
  echo "\"global_var\" in function = $global_var"
}

func

# Now, to see if local variable "loc_var" exists outside the function.
echo
echo "\"loc_var\" outside function = $loc_var"
                                   # $loc_var outside function =
                                   # No, $loc_var not visible globally.

echo "\"global_var\" outside function = $global_var"
                                      # $global_var outside function = 999
                                      # $global_var is visible globally.

exit 0

#  In contrast to C, a Bash variable declared inside a function
#+ is local ONLY if declared as such.


In [4]:
bash ex62.sh


"loc_var" in function = 23
"global_var" in function = 999

"loc_var" outside function = 
"global_var" outside function = 999


Before a function is called, all variables declared within the function are invisible outside the body of the function, not just those explicitly declared as local.

In [5]:
cat invar.sh

#!/bin/bash

func ()
{
  global_var=37        #  Visible only within the function block
                       #+ before the function has been called.
}                      # END OF FUNCTION

echo "global_var = $global_var"   # global_var =
                                  #  Function "func" has not yet been called,
                                  #+ so $global_var is not visible here.

func

echo "global_var = $global_var"   # global_var = 37
                                  # Has been set by function call.


In [6]:
bash invar.sh

global_var = 
global_var = 37


As Evgeniy Ivanov points out, when declaring and setting a local variable in a single command, apparently the order of operations is to first set the variable, and only afterwards restrict it to local scope. This is reflected in the return value.

In [7]:
cat var.sh

#!/bin/bash

echo "==OUTSIDE Function (global)=="
t=$(exit 1)
echo $?      # 1
             # As expected.

echo

function0 ()
{

  echo "==INSIDE Function=="
  echo "Global"
  t0=$(exit 1)
  echo $?     # 1
              # As expected.

  echo
  echo "Local declared & assigned in same command."
  local t1=$(exit 1)
  echo $?     # 0
              # Unexpected!
  # Apparently, the variable assignment takes place before
  #+ the local declaration.
  #+ The return value is for the latter.
  echo
  echo "Local declared, then assigned (separate commands)."

  local t2
  t2=$(exit 1)
  echo $?     # 1
              # As expected.
}

function0


In [8]:
bash var.sh

==OUTSIDE Function (global)==
1

==INSIDE Function==
Global
1

Local declared & assigned in same command.
0

Local declared, then assigned (separate commands).
1


#### Local variables and recursion

Recursion is an interesting and sometimes useful form of self-reference.   
Herbert Mayer defines it as ". . . expressing an algorithm by using a simpler version of that same algorithm . . ."

Consider a definition defined in terms of itself, an expression implicit in its own expression, a snake swallowing its own tail, or . . . a function that calls itself.

#### Example 24-13. Demonstration of a simple recursive function

In [9]:
cat recursion-demo.sh

#!/bin/bash
# recursion-demo.sh
# Demonstration of recursion.

RECURSIONS=9    # How many times to recurse.
r_count=0       # Must be global. Why?

recurse ()
{
  var="$1"

  while [ "$var" -ge 0 ]
  do
    echo "Recursion count = "$r_count" +-+ \$var = "$var""
    (( var-- )); (( r_count++ ))
    recurse "$var" #  Function calls itself (recurses)
  done             #+ until what condition is met?
}

recurse $RECURSIONS

exit $?


In [10]:
bash recursion-demo.sh

Recursion count = 0 +-+ $var = 9
Recursion count = 1 +-+ $var = 8
Recursion count = 2 +-+ $var = 7
Recursion count = 3 +-+ $var = 6
Recursion count = 4 +-+ $var = 5
Recursion count = 5 +-+ $var = 4
Recursion count = 6 +-+ $var = 3
Recursion count = 7 +-+ $var = 2
Recursion count = 8 +-+ $var = 1
Recursion count = 9 +-+ $var = 0


#### Example 24-14. Another simple demonstration

In [11]:
cat recursion-def.sh

#!/bin/bash
# recursion-def.sh
# A script that defines "recursion" in a rather graphic way.

RECURSIONS=10
r_count=0
sp=" "

define_recursion ()
{
  ((r_count++))
  sp="$sp"" "
  echo -n "$sp"
  echo "\"The act of recurring ... \""   # Per 1913 Webster's dictionary.
  while [ $r_count -le $RECURSIONS ]
  do
    define_recursion
  done
}

echo
echo "Recursion: "
define_recursion
echo

exit $?


In [12]:
bash recursion-def.sh


Recursion: 
  "The act of recurring ... "
   "The act of recurring ... "
    "The act of recurring ... "
     "The act of recurring ... "
      "The act of recurring ... "
       "The act of recurring ... "
        "The act of recurring ... "
         "The act of recurring ... "
          "The act of recurring ... "
           "The act of recurring ... "
            "The act of recurring ... "



Local variables are a useful tool for writing recursive code, but this practice generally involves a great deal of computational overhead and is definitely not recommended in a shell script.

#### Example 24-15. Recursion, using a local variable

In [13]:
cat recursion-local.sh

#!/bin/bash
#             factorial
#              ---------

# Does bash permit recursion?
# Well, yes, but...
# It's so slow that you gotta have rocks in your head to try it.

MAX_ARG=5
E_WRONG_ARGS=85
E_RANGE_ERR=86

if [ -z "$1" ]
then
  echo "Usage: `basename $0` number"
  exit $E_WRONG_ARGS
fi

if [ "$1" -gt $MAX_ARG ]
then
  echo "Out of range ($MAX_ARG is maximum)."
  #  Let's get real now.
  #  If you want greater range than this,
  #+ rewrite it in a Real Programming Language.
  exit $E_RANGE_ERR
fi

fact ()
{
  local number=$1
  #  Variable "number" must be declared as local,
  #+ otherwise this doesn't work.
  if [ "$number" -eq 0 ]
  then
    factorial=1    # Factorial of 0 = 1.
  else
    let "decrnum = number - 1"
    fact $decrnum  # Recursive function call (the function calls itself).
    let "factorial = $number * $?"
  fi
  return $factorial
}

fact $1
echo "Factorial of $1 is $?."

exit 0


In [14]:
bash recursion-local.sh 5

Factorial of 5 is 120.


Also see Example A-15 for an example of recursion in a script.   
Be aware that recursion is resource-intensive and executes slowly,   
and is therefore generally not appropriate in a script.

### Recursion Without Local Variables

A function may recursively call itself even without use of local variables.

#### Example 24-16. The Fibonacci Sequence

In [15]:
cat fibo.sh

#!/bin/bash
# fibo.sh : Fibonacci sequence (recursive)
# Author: M. Cooper
# License: GPL3

# ----------algorithm--------------
# Fibo(0) = 0
# Fibo(1) = 1
# else
#   Fibo(j) = Fibo(j-1) + Fibo(j-2)
# ---------------------------------

MAXTERM=15      # Number of terms (+1) to generate.
MINIDX=2        # If idx is less than 2, then Fibo(idx) = idx.

Fibonacci ()
{
  idx=$1        # Doesn't need to be local. Why not?
  if [ "$idx" -lt "$MINIDX" ]
  then
    echo "$idx" # First two terms are 0 1 ... see above.
  else
    (( --idx )) # j-1
    term1=$( Fibonacci $idx ) # Fibo(j-1)
    (( --idx )) # j-2
    term2=$( Fibonacci $idx ) # Fibo(j-2)
    echo $(( term1 + term2 ))
  fi

  #  An ugly, ugly kludge.
  #  The more elegant implementation of recursive fibo in C
  #+ is a straightforward translation of the algorithm in lines 7 - 10.
}

for i in $(seq 0 $MAXTERM)
do  # Calculate $MAXTERM+1 terms.
  FIBO=$(Fibonacci $i)
  echo -n "$FIBO "
done
# 0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 61

In [16]:
bash fibo.sh

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 


#### Example 24-17. The Towers of Hanoi

In [17]:
cat hanoi.sh

#! /bin/bash
#
# The Towers Of Hanoi
# Bash script
# Copyright (C) 2000 Amit Singh. All Rights Reserved.
# http://hanoi.kernelthread.com
#
# Tested under Bash version 2.05b.0(13)-release.
# Also works under Bash version 3.x.
#
#  Used in "Advanced Bash Scripting Guide"
#+ with permission of script author.
#  Slightly modified and commented by ABS author.

#=================================================================#
#  The Tower of Hanoi is a mathematical puzzle attributed to
#+ Edouard Lucas, a nineteenth-century French mathematician.
#
#  There are three vertical posts set in a base.
#  The first post has a set of annular rings stacked on it.
#  These rings are disks with a hole drilled out of the center,
#+ so they can slip over the posts and rest flat.
#  The rings have different diameters, and they stack in ascending
#+ order, according to size.
#  The smallest ring is on top, and the largest on the bottom.
#
#  The task is to transfer the stack of rings
#+ to one of the oth

In [18]:
bash hanoi.sh 5

move 1 --> 3
move 1 --> 2
move 3 --> 2
move 1 --> 3
move 2 --> 1
move 2 --> 3
move 1 --> 3
move 1 --> 2
move 3 --> 2
move 3 --> 1
move 2 --> 1
move 3 --> 2
move 1 --> 3
move 1 --> 2
move 3 --> 2
move 1 --> 3
move 2 --> 1
move 2 --> 3
move 1 --> 3
move 2 --> 1
move 3 --> 2
move 3 --> 1
move 2 --> 1
move 2 --> 3
move 1 --> 3
move 1 --> 2
move 3 --> 2
move 1 --> 3
move 2 --> 1
move 2 --> 3
move 1 --> 3
Total moves = 31
